In [ ]:
import cv2
import numpy as np
yolo_weights = "yolov4.weights"  
yolo_config = "yolov4.cfg"
classes_file = "coco.names"
with open(classes_file, "r") as f:
    class_names = [line.strip() for line in f.readlines()]
net = cv2.dnn.readNet(yolo_weights, yolo_config)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
vehicle_classes = {"car", "bus", "truck", "motorbike"}
def count_vehicles_and_control_signal(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found at {image_path}")   
    height, width, _ = image.shape
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward(output_layers)

    vehicle_count = 0 

    for output in detections:
        for detection in output:
            scores = detection[5:]  
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_names[class_id] in vehicle_classes:
                vehicle_count += 1
                center_x, center_y, box_width, box_height = (
                    detection[0:4] * np.array([width, height, width, height])
                ).astype("int")
                x = int(center_x - box_width / 2)
                y = int(center_y - box_height / 2)
                label = f"{class_names[class_id]}: {confidence:.2f}"
                cv2.rectangle(image, (x, y), (x + int(box_width), y + int(box_height)), (0, 255, 0), 2)
                cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    if vehicle_count > 60:
        green_light_time = "90 seconds"
    elif 55 < vehicle_count <= 60:
        green_light_time = "70 seconds"
    elif 45 < vehicle_count <= 50:
        green_light_time = "60 seconds"
    elif 35 < vehicle_count <= 40:
        green_light_time = "50 seconds"
    elif 25 < vehicle_count <= 30:
        green_light_time = "40 seconds"
    elif 15 < vehicle_count <= 20:
        green_light_time = "30 seconds"
    elif 5 < vehicle_count <= 10:
        green_light_time = "20 seconds"
    else:
        green_light_time = "10 seconds"
    cv2.putText(image, f"Vehicles: {vehicle_count}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(image, f"Green Light: {green_light_time}", (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    return image, vehicle_count, green_light_time
image_path = "traffic1.jpg" 

try:
    annotated_image, vehicle_count, green_light_time = count_vehicles_and_control_signal(image_path)
    print(f"Vehicles Detected: {vehicle_count}")
    print(f"Green Light Duration: {green_light_time}")

    cv2.imshow("Vehicle Detection and Signal Control", annotated_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    cv2.imwrite("output.jpg", annotated_image)

except ValueError as e:
    print(e)

Vehicles Detected: 38
Green Light Duration: 50 seconds
